## Set up the dependencies
When running in a distributed Ray Cluster, all nodes need to have access to dependencies. For this, we'll use `pip install --user` to install the necessary requirements. On an Anyscale Workspace, this is configured to install packages to a shared filesystem that will be available to all nodes in the cluster.

`pip install --user -r requirements.txt`  

After installing all the requirements, we'll start with some imports.

In [ ]:
!pip install --user -r requirements.txt

In [1]:
import cv2
import ray

In [ ]:
video = cv2.VideoCapture('./car_videos/cars.mp4')
fps = video.get(cv2.CAP_PROP_FPS)
print('frames per second =',fps)
# minutes = 0
# seconds = 1
# frame_id = int(fps*(minutes*60 + seconds))
# print('frame id =',frame_id)

In [ ]:
success,image = video.read()
count = 0
print(success)
while success:
    if count %10 ==0:
        cv2.imwrite("./car_images/frame%d.jpg" % count, image)     # save frame as JPEG file      
        print('write a frame: ', count)
    success,image = video.read()
    count += 1

In [ ]:
from img2img import PredictCallable

In [ ]:
model_id = 'nitrosocke/Ghibli-Diffusion'
#model can be saved under /mnt/shared_storage for re-use
#model_id = '/mnt/shared_storage/kyle/huggingface/diffusers/models--nitrosocke--Ghibli-Diffusion/snapshots/441c2fde3e82de3720b1a35111b95b6d8afee9d6/'
img_path = './car_images'
inp_images = ray.data.read_images(img_path, size=(768, 768), include_paths=True).limit(20)
preds = (
    inp_images
    .map_batches(
        PredictCallable,
        batch_size=2,
        fn_constructor_kwargs=dict(model_id=model_id, prompt='cyberpunk style cars', strength=0.95),
        compute="actors",
        num_gpus=1
    )
)

In [ ]:
res = preds.take_all()

In [ ]:
print(len(res))
if len(res) < 20:
    for re in res:
        for k in re.keys():
            print(k)
            display(re[k])

In [ ]:
import numpy as np
frameSize = (768, 768)

out = cv2.VideoWriter('car_videos/output_video.avi', 0, 1, frameSize)

for re in res:
    for k in re.keys():
        print(k,type(re[k]))
        pil_image = re[k]
        open_cv_image = np.array(pil_image) 
        # Convert RGB to BGR 
        open_cv_image = open_cv_image[:, :, ::-1].copy() 
        print(open_cv_image.shape)
        out.write(open_cv_image)